In [2]:
import psycopg2

try:
    connection = psycopg2.connect(
        user="user",
        port="port",
        host="localhost",
        database="name"
    )
    
    cursor = connection.cursor()
    
    # The table sentiment_analysis_data_aapl contains tens of thousands of news articles about AAPL
    select_query1 = "SELECT * FROM sentiment_analysis_data_aapl;"
    
    # Execute the query
    cursor.execute(select_query1)
    # Fetch all records
    records_aapl = cursor.fetchall()
    print(records_aapl[700][3])
    
except Exception as error:
    print(f"Failed to fetch records: {error}")

finally:
    # Close the connection
    if connection:
        cursor.close()
        connection.close()
        print("Connection closed.")

This column was originally published on RealMoney on Jan. 18 at 8:42 a.m. ET. It's being republished as a bonus for TheStreet.com readers. For more information about subscribing to RealMoney, please click here. Let's talk about sentiment. More specifically, let's discuss the Investors Intelligence readings that came out Wednesday. The percentage of bulls sank to 50.5%, well off the high of 59.8% in early December. Yes, this confirms that the bullish ranks are thinning. But please, do not confuse this with too much bearishness. There's not too much bearishness -- just less bullishness. The percentage of bears in the survey has remained steadily in the 20% range for months now, so anyone who wants to fuss over a rise of 2 percentage points is grasping at straws, in my opinion. However, the massive rise in the percentage of folks looking for a correction was noteworthy. We last discussed this number in early December. I went through great pains to show that when this reading gets below 17

In [ ]:
# Connect to database
connection = psycopg2.connect(
    user="user",
    port="port",
    host="localhost",
    database="name"
)

cursor = connection.cursor()

drop_table_query = "DROP TABLE IF EXISTS sentiment_data_aapl;"
cursor.execute(drop_table_query)
connection.commit()
print("Table dropped successfully.")

create_table_query = '''CREATE TABLE sentiment_data_aapl (
    ID SERIAL PRIMARY KEY,
    Title TEXT,
    Time_Published TEXT,
    Sentiment_Score REAL
); '''

cursor.execute(create_table_query)
connection.commit()
print("Table Created Successfully")

Table dropped successfully.
Table Created Successfully


In [14]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from nltk.tokenize import sent_tokenize
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

connection = psycopg2.connect(
    user="user",
    port="port",
    host="localhost",
    database="name"
)

cursor = connection.cursor()

# Load the model and tokenizer
model_name = "nickmuchi/deberta-v3-base-finetuned-finance-text-classification"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)  # Moved to device
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocessing function
def preprocess(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    return inputs

# Inference function for a single sentence
def get_sentiment(text):
    inputs = preprocess(text).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    label_id = torch.argmax(probabilities).item()
    return label_id

# Your articles
article_text = [record[3] for record in records_aapl]
article_date = [record[2] for record in records_aapl]
article_title = [record[1] for record in records_aapl]

# Iterate over articles
for i, (article, date, title) in enumerate(zip(article_text, article_date, article_title)):
    sentences = sent_tokenize(article)  # Split the article into sentences
    sentiment_scores = [get_sentiment(sentence) for sentence in sentences]  # Get sentiment for each sentence
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores)  # Average the sentiment scores
    cursor.execute("INSERT INTO sentiment_data_aapl (ID, Title, Sentiment_Score, Time_Published) VALUES (%s, %s, %s, %s)",
                   (i, title, avg_sentiment, date))
    connection.commit()
    print(f"Average Sentiment of Article {i+1} is {avg_sentiment} at {date} for {title}")


Average Sentiment of Article 1 is 1.1111111111111112 at January 12, 2007 Friday 10:40 AM EST
Average Sentiment of Article 2 is 1.1111111111111112 at January 16, 2007 Tuesday 8:46 PM EST
Average Sentiment of Article 3 is 1.1111111111111112 at January 17, 2007 Wednesday 12:55 PM EST
Average Sentiment of Article 4 is 1.1111111111111112 at January 15, 2007 Monday 6:26 PM EST
Average Sentiment of Article 5 is 1.1111111111111112 at January 16, 2007 Tuesday 7:25 PM EST
Average Sentiment of Article 6 is 1.2972972972972974 at January 1, 2007 Monday 11:55 AM EST
Average Sentiment of Article 7 is 1.0 at January 30, 2007 Tuesday 08:54 AM EST
Average Sentiment of Article 8 is 0.4583333333333333 at January 19, 2007 Friday 9:16 AM EST
Average Sentiment of Article 9 is 1.0158730158730158 at January 17, 2007 Wednesday 9:30 AM EST
Average Sentiment of Article 10 is 1.0588235294117647 at January 19, 2007 Friday 09:26 AM EST
Average Sentiment of Article 11 is 0.9183673469387755 at January 22, 2007 Monday 

KeyboardInterrupt: 